In [183]:
# CODING STYLE: https://peps.python.org/pep-0008/
from dataclasses import dataclass, asdict 
from asciitree import LeftAligned
from asciitree.drawing import BOX_DOUBLE, BoxStyle
from typing import Union
from datetime import datetime
import warnings 
from astroquery.heasarc import Heasarc, Conf
import astropy.units as u
from astropy.coordinates import SkyCoord
import pandas as pd 
import numpy as np

heasarc = Heasarc()
Conf.server.set('https://www.isdc.unige.ch/browse/w3query.pl')

def _to_print_dict(conf):
    out = {}

    if not isinstance(conf, dict):

        dummy = {}
        dummy[f"{conf}"] = {}

        return dummy
    
    else:
        for k, v in conf.items():

            if not isinstance(v, dict):

                text = f"{k}" ##############################################################################

            else:

                text = f"{k}"

            out[text] = _to_print_dict(v)

    return out


class BaseDataClass:
    def show(self):
        final = {}
        final[type(self).__name__] = _to_print_dict(asdict(self))
        tr = LeftAligned(draw=BoxStyle(gfx=BOX_DOUBLE, horiz_len=1))
        print(tr(final))

        
@dataclass
class Range(BaseDataClass):
    """
    Simple dataclass that represents a range
    """
    min_val: Union[str, float, None] = None 
    max_val: Union[str, float, None] = None 
    
    
@dataclass
class Filter(BaseDataClass):
    """
    Dataclass to handle the filter parameters
    """
    SCW_ID: Union[str, None] = None 
    SCW_VER: Union[int, None] = None 
    SCW_TYPE: Union[int, None] = None 
    RA_X: Union[Range, None] = None
    DEC_X: Union[Range, None] = None
    TIME: Union[Range, None] = None
    #END_DATE: Union[Range, None] = None
    OBS_ID: Union[str, None] = None
    OBS_TYPE: Union[str, None]  = None
    PS: Union[str, None]  = None
    PI_NAME: Union[str, None]  = None
    GOOD_SPI: Union[Range, None]  = None
    GOOD_PICSIT: Union[Range, None] = None
    GOOD_JMEX: Union[Range, None] = None
    GOOD_JMEX1: Union[Range, None] = None
    GOOD_JMEX2: Union[Range, None] = None
    GOOD_OMC: Union[Range, None] = None
    DSIZE: Union[Range, None] = None
    _SEARCH_OFFSET: Union[Range, None] = None
    
    
@dataclass
class SearchQuery(BaseDataClass):
    """
    Dataclass to handle the inital search query parameters
    """
    object_name: Union[str, None] = None
    position: Union[str, None] = None
    radius: Union[int, None] = None
    mission: Union[str, None] = "integral_rev3_scw"
    sortvar: Union[str, None] = "START_DATE"
    resultmax: Union[int, None] = 0
    
    @property
    def object_dict(self):
        """
        Get dict for the object modus => drop radius and position
        :returns:
        """
        dic = asdict(self)
        dic.pop("radius")
        dic.pop("position")
        return dic
    
    @property
    def region_dict(self):
        """
        Get dict for the region modus => drop object_name
        :returns:
        """
        dic = asdict(self)
        dic.pop("object_name")
        return dic
    

class IntegralQuery:
    def __init__(self, search_query: SearchQuery):
        """
        Init the Integral query object. Used to get the SCW_ID for a certain position or
        object and apply different filters to it
        """
        assert (search_query.object_name is not None or 
                (search_query.position is not None and 
                 search_query.radius is not None)), "Please specify either object_name, or position and radius"
        
        if search_query.object_name:
            self._table = heasarc.query_object(**search_query.object_dict)

        else:
            self._table = heasarc.query_region(**search_query.region_dict)
            
        self._format_table()

    def _format_table(self):
        """
        Format the table
        :returns:
        """
        # BB: self._table
        self._table.convert_bytestring_to_unicode()
        self._table = self._table.to_pandas()

        int_columns = ["SCW_VER"]
        float_columns = ["RA_X", "DEC_X", "GOOD_SPI", "GOOD_PICSIT", "GOOD_ISGRI", 
                         "GOOD_JEMX", "GOOD_JEMX1", "GOOD_JEMX2", "GOOD_OMC", "DSIZE"]
        string_columns = ["SCW_ID", "SCW_TYPE", "OBS_TYPE","OBS_ID", "PS", "PI_NAME"]
        datetime_columns = ["START_DATE", "END_DATE"]
        
        for c in int_columns:
            self._table[c] = self._table[c].astype(int)
            
        for c in float_columns:
            self._table[c] = self._table[c].str.strip()
            mask = self._table.copy()[c]==""
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                self._table[c].loc[mask] = 0
            self._table[c] = self._table[c].astype(float) ####################################################################
            
        for c in string_columns:
            self._table[c] = self._table[c].str.strip()
            
        for c in datetime_columns:
            self._table[c] = pd.to_datetime(self._table[c])

    def sort_by(self, sortvar):
        """
        Sort the tables by a new Variable
        :param sortvar: Variable to use for the sort
        :returns:
        """
        self._table = self._table.sort_values(sortvar)

    def apply_filter(self, filter_ob: Filter, return_coordinates=False) -> np.array:
        """
        Apply a filter to the base table
        :param filter_ob: Filter Object with all the filter parameters
        :param return_coordintes: Specifies if coordintes should be returned
        :returns:
        """
        new_table = self._table.copy()
        for key, value in asdict(filter_ob).items():
            if value:
                if type(value) is dict:
                    if key=="TIME":
                        if value["min_val"]:
                            new_table = new_table[new_table["START_DATE"]>=
                                                  datetime.fromisoformat(value["min_val"])]
                        if value["max_val"]:
                            new_table = new_table[new_table["END_DATE"]<=
                                                  datetime.fromisoformat(value["max_val"])]
                    else:
                        if value["min_val"]:
                            new_table = new_table[new_table[key]>=value["min_val"]]
                        if value["max_val"]:
                            new_table = new_table[new_table[key]<=value["max_val"]]
                else:
                    new_table = new_table[new_table[key]==value]
        if not return_coordinates:
            return new_table["SCW_ID"].to_numpy()
        else:
            # print(type(new_table["START_DATE"].iloc[1]))
            # new_table["START_DATE"] = new_table["START_DATE"].dt.to_pydatetime() ################################################
            # print(type(new_table["START_DATE"].iloc[1]))
            return np.concatenate((new_table[["SCW_ID","RA_X","DEC_X"]].to_numpy(), new_table["START_DATE"].dt.to_pydatetime().reshape(-1)), axis=0) #############################################################
    
    @property
    def table(self):
        """
        :returns: Base Table of Query
        """
        return self._table

In [163]:
searchquerry = SearchQuery(object_name="Cyg X-1", resultmax=0)
searchquerry.show()

SearchQuery
 ╠═ object_name
 ║  ╚═ Cyg X-1
 ╠═ position
 ║  ╚═ None
 ╠═ radius
 ║  ╚═ None
 ╠═ mission
 ║  ╚═ integral_rev3_scw
 ╠═ sortvar
 ║  ╚═ START_DATE
 ╚═ resultmax
    ╚═ 0


In [164]:
cat = IntegralQuery(searchquerry)
cat.table


,SCW_ID,SCW_VER,SCW_TYPE,RA_X,DEC_X,START_DATE,END_DATE,OBS_ID,OBS_TYPE,PS,PI_NAME,GOOD_SPI,GOOD_PICSIT,GOOD_ISGRI,GOOD_JEMX,GOOD_JEMX1,GOOD_JEMX2,GOOD_OMC,DSIZE,_SEARCH_OFFSET
0,001130000050,1,POINTING,299.543488,35.301971,2002-11-16 20:06:11,2002-11-16 20:16:53,,,PUBLIC,,575.0,0.0,0.0,571.0,571.0,571.0,575.0,13746176.0,6.444
1,001130000061,1,SLEW,299.567740,35.250057,2002-11-16 20:16:53,2002-11-16 20:18:23,,,PUBLIC,,89.0,1.0,12.0,75.0,74.0,75.0,90.0,3153920.0,3.111
2,001130000070,1,POINTING,299.590179,35.201195,2002-11-16 20:18:23,2002-11-16 20:55:43,00600030001,CALIBRATION,PUBLIC,PUBLIC,1861.0,1815.0,1330.0,2221.0,2212.0,2221.0,2196.0,69844992.0,0.026
3,001130000080,1,POINTING,299.590179,35.201195,2002-11-16 20:55:43,2002-11-16 21:53:21,00600030001,CALIBRATION,PUBLIC,PUBLIC,3455.0,3432.0,2161.0,3456.0,3456.0,2704.0,3458.0,117604352.0,0.026
4,001130000091,1,SLEW,300.101460,36.279747,2002-11-16 21:53:21,2002-11-16 21:55:17,00600030001,CALIBRATION,PUBLIC,PUBLIC,116.0,0.0,72.0,116.0,116.0,0.0,0.0,4890624.0,69.312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19161,250300240010,1,POINTING,306.328033,35.514473,2022-05-16 13:17:35,2022-05-16 14:13:10,19200110001,GENERAL,PRIVATE,PROF.\nJOERN\nWILMS,3297.0,3271.0,3296.0,3231.0,3231.0,3156.0,0.0,146329600.0,330.167
19162,250300240021,1,SLEW,307.154813,34.567347,2022-05-16 14:13:10,2022-05-16 14:15:05,19200110001,GENERAL,PRIVATE,PROF.\nJOERN\nWILMS,114.0,0.0,115.0,113.0,113.0,0.0,0.0,5840896.0,374.160
19163,250300250010,1,POINTING,307.853790,33.744278,2022-05-16 14:15:05,2022-05-16 15:10:39,19200110001,GENERAL,PRIVATE,PROF.\nJOERN\nWILMS,3274.0,3247.0,3273.0,3274.0,3274.0,3274.0,0.0,144752640.0,417.852
19164,250300250021,1,SLEW,307.185148,33.354976,2022-05-16 15:10:39,2022-05-16 15:12:35,19200110001,GENERAL,PRIVATE,PROF.\nJOERN\nWILMS,115.0,0.0,116.0,116.0,116.0,114.0,0.0,6094848.0,392.383


In [165]:
type(cat.table["START_DATE"][1])

pandas._libs.tslibs.timestamps.Timestamp

In [166]:
f = Filter(SCW_TYPE="POINTING", TIME=Range(min_val='2009-12-13T00:05:23', max_val='2015-09-04T00:05:23'), RA_X=Range(304.,306.))
f.show()

Filter
 ╠═ SCW_ID
 ║  ╚═ None
 ╠═ SCW_VER
 ║  ╚═ None
 ╠═ SCW_TYPE
 ║  ╚═ POINTING
 ╠═ RA_X
 ║  ╠═ min_val
 ║  ║  ╚═ 304.0
 ║  ╚═ max_val
 ║     ╚═ 306.0
 ╠═ DEC_X
 ║  ╚═ None
 ╠═ TIME
 ║  ╠═ min_val
 ║  ║  ╚═ 2009-12-13T00:05:23
 ║  ╚═ max_val
 ║     ╚═ 2015-09-04T00:05:23
 ╠═ OBS_ID
 ║  ╚═ None
 ╠═ OBS_TYPE
 ║  ╚═ None
 ╠═ PS
 ║  ╚═ None
 ╠═ PI_NAME
 ║  ╚═ None
 ╠═ GOOD_SPI
 ║  ╚═ None
 ╠═ GOOD_PICSIT
 ║  ╚═ None
 ╠═ GOOD_JMEX
 ║  ╚═ None
 ╠═ GOOD_JMEX1
 ║  ╚═ None
 ╠═ GOOD_JMEX2
 ║  ╚═ None
 ╠═ GOOD_OMC
 ║  ╚═ None
 ╠═ DSIZE
 ║  ╚═ None
 ╚═ _SEARCH_OFFSET
    ╚═ None


In [167]:
scw_ids = cat.apply_filter(f,True)

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 1 dimension(s)

In [ ]:
scw_ids

array([['087500190010', 304.286499, 36.094223,
        Timestamp('2009-12-13 00:59:36')],
       ['087500640010', 304.946136, 43.962917,
        Timestamp('2009-12-14 02:20:48')],
       ['087500740010', 305.695282, 40.948917,
        Timestamp('2009-12-14 08:14:53')],
       ...,
       ['156300280010', 304.511749, 31.972778,
        Timestamp('2015-07-12 18:20:09')],
       ['156300300010', 305.668335, 35.295444,
        Timestamp('2015-07-12 20:11:09')],
       ['156300380010', 305.684723, 33.100834,
        Timestamp('2015-07-13 03:39:26')]], dtype=object)

In [ ]:
type(scw_ids)

numpy.ndarray

In [ ]:
type(scw_ids[0,3])

pandas._libs.tslibs.timestamps.Timestamp

In [ ]:
a=np.zeros((2,3))
a

array([[0., 0., 0.],
       [0., 0., 0.]])

In [179]:
b=np.array([np.zeros(2)]).T
b

array([[0.],
       [0.]])

In [181]:
np.concatenate((a,b),axis=1)

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.]])